# PLAYGROUND for AgentGENius

In [1]:
from datetime import date

import nest_asyncio
from dotenv import load_dotenv
from pydantic_ai import RunContext, Tool

from agentgenius.agents import AgentStore, BaseAgent
from agentgenius.config import config
from agentgenius.core import AgentGENius
from agentgenius.tools import ToolSet

nest_asyncio.apply()
load_dotenv()

True

## Testing AgentStore 

In [2]:
ags = AgentStore(config.agents_path).load_agents()
# ags.add(AgentGENius.from_file(config.agents_path / "assistant.json"))
print(ags.list())

if agent := ags["assistant"]:
    print(agent.run_sync("test").data)

Test received! How can I assist you today?


In [11]:
agent = AgentGENius.from_file(config.agents_path / "assistant.json")


@agent.tool_plain
def ask_user_tool(question: str) -> str:
    """Ask the user a question"""
    return input(question + " ")


result = agent.run_sync("is today my birthday?")
print(agent.to_json())
print(result.data)

{"name":"assistant","model":"openai:gpt-4o-mini","system_prompt":"You are a helpful AI assistant.","toolset":[]}
Today is not your birthday. Happy birthday for when it comes!


In [12]:
def test_tool(value: int) -> str:
    return str(value)


tools = ToolSet(test_tool)

tool = tools.get("test_tool")
assert tool(10) == "10"

assert "test_tool2" not in tools
assert "test_tool" in tools
assert test_tool in tools

tools.remove("test_tool")
tool = tools.get("test_tool")
assert tool is None

In [13]:
def ask_user_tool(ctx: RunContext[str], question: str) -> str:
    """Ask the user a question"""
    return input(question + " ")


tools = ToolSet(ask_user_tool)

agent = AgentGENius(
    name="assistant", model="openai:gpt-4o-mini", system_prompt="You are a helpful assistant.", toolset=tools
)


@agent.system_prompt
def add_the_date() -> str:
    return f"The date is {date.today()}."


print(f"{agent.to_json()=}")

j = agent.to_json()
agent2 = BaseAgent.from_json(j)


print(f"{agent2.to_json()=}")
## Agents tolsets aren't the same, not working in the notebook for some reason

agent.to_json()='{"name":"assistant","model":"openai:gpt-4o-mini","system_prompt":"You are a helpful assistant.","toolset":["ask_user_tool"]}'
agent2.to_json()='{"name":"assistant","model":"openai:gpt-4o-mini","system_prompt":"You are a helpful assistant.","toolset":[]}'
